In [1]:
import os

In [2]:
os.listdir(".")

['Care_review_full.csv',
 'pubmed_1k.json',
 'test_pca.json',
 'Care_Reviews_10k.json',
 'BertBasedUncasedWordEmbAverage.ipynb',
 'test.csv',
 'submission.csv',
 'test_cleaned.csv',
 'test_lda.csv',
 'bert_viz_phase2.ipynb',
 'news_50k_cleaned.csv',
 'news_50k_test_lda.csv',
 'Care_Reviews_10k.csv',
 '.ipynb_checkpoints',
 'news_50k_test_embs.json',
 'news_50k_test.csv',
 'news_50k.json',
 'bert_viz_phase2-Copy3.ipynb']

In [4]:
# Convert .json to .csv
# Adds the required columns with reindexing
# Adds data category based on rating in data title

import pandas as pd
with open('Care_Reviews_10k.json', encoding='utf-8') as inputfile:
    df = pd.read_json(inputfile)
df
df = df.drop(columns=[1,3,4,6])
df = df.rename(columns={0:'data_id', 2:'data_title', 5:'data_string'})

df['2d_coor'] = ''
df['data_category'] = ''
df['data_category_number'] = ''
df = df.reindex(columns=['data_id', 'data_string', '2d_coor', 'data_title', 'data_category', 'data_category_number'])
df.dropna()

for index in range(len(df)):
    pos=df.loc[index, "data_title"].find("rating: ")
    #print(pos)
   
    #print(aaa)
    df['data_category_number'][index]=df.loc[index, "data_title"][pos+8]
    if df.loc[index, "data_title"][pos+8] == "1":
        df['data_category'][index]="Poor"
    elif df.loc[index, "data_title"][pos+8] == "2":
        df['data_category'][index]="Fair"
    elif df.loc[index, "data_title"][pos+8] == "3":
        df['data_category'][index]="Good"
    elif df.loc[index, "data_title"][pos+8] == "4":
        df['data_category'][index]="Very Good"
    elif df.loc[index, "data_title"][pos+8] == "5":
        df['data_category'][index]="Excelent"
    df.loc[index, "data_title"]=df.loc[index, "data_title"][0:pos-2]
df.head()

df.to_csv('Care_Reviews_10k.csv', encoding='utf-8', index=False)
display(df)

,data_id,data_string,2d_coor,data_title,data_category,data_category_number
0,a224545f-1a07-47eb-96ee-a0d0cab23100,Good place to visit,,urgentCare: Access Urgent Medical Care Pickeri...,Very Good,4
1,01a8cfc8-e207-4c72-a9fa-fa6353fde529,Went here for a swollen Jaw. Even though I was...,,urgentCare: Access Urgent Medical Care Pickeri...,Poor,1
2,1a4fb4a6-6c56-4610-a151-b4b25a774cc8,I was seen relatively quickly and the staff wa...,,urgentCare: Access Urgent Medical Care Pickeri...,Excelent,5
3,9356a4fd-54fd-4604-a6a8-2d3067eba7cc,Reception and service couldn't have been more ...,,urgentCare: Access Urgent Medical Care Pickeri...,Excelent,5
4,77526eed-5ff3-4f2f-80ce-da1048137cf4,I came in they were very busy the receptionist...,,urgentCare: Access Urgent Medical Care Pickeri...,Excelent,5
...,...,...,...,...,...,...
9995,0669d71c-f596-4183-a6ef-6ceb660cac3f,I needed a prescription and thought going to t...,,urgentCare: Cleveland Clinic Express Care Clinic,Poor,1
9996,7a73d727-b0eb-48e2-8c01-b6faf07ea776,"Excellent, friendly staff. Got me in and out i...",,urgentCare: CareFirst Urgent Care - Kenwood,Excelent,5
9997,baf90288-9dc4-4de4-9b8f-f31322f48747,Awesome staff!!,,urgentCare: CareFirst Urgent Care - Kenwood,Excelent,5
9998,672cf0b1-0bc6-45aa-be18-b80d196870f7,I'm really glad that CareFirst open this facil...,,urgentCare: CareFirst Urgent Care - Kenwood,Excelent,5


In [15]:
!pip install simpletransformers

In [2]:
# Uses "bert-base-uncased" to get embeddings
# For words more than 500, divides in chunks and gets average embeddings
# Prints the rows were we have more than 500 words and averaging of embeddings needs to be done
# Takes time to print on local GPU
import transformers
import torch
import numpy as np
from simpletransformers.language_representation import RepresentationModel

model=RepresentationModel(model_type="bert", model_name="bert-base-uncased", use_cuda=False)

def updateDataFrame2(csv_file_name='Care_Reviews_10k.csv', model_max_length=500):
    import pandas as pd
    data = pd.read_csv(csv_file_name)
    data=data.astype({'2d_coor': 'object'})
    
    def split_sentences(list_of_words):
        chunked_list = list()
        chunk_size = model_max_length
        for i in range(0, len(list_of_words), chunk_size):
            chunked_list.append(list_of_words[i:i+chunk_size])

        return(chunked_list)

    for idx in range(len(data['data_string'])):
        chunkedListWithSent=[]
        sent=""
    
        if len(str(data['data_string'][idx]).split()) > 500:
            tempString = data['data_string'][idx]
            tempStringSplit = tempString.split()
            chunkedLists = split_sentences(tempStringSplit)
            
            for sentenceList in chunkedLists:
                sent=" ".join(sentenceList)
                chunkedListWithSent.append(sent)
                sent=""
             
            sentenceVectors = model.encode_sentences(chunkedListWithSent, combine_strategy="mean")
            word_embedding_avg = np.mean(sentenceVectors, axis=0)

            print(idx)
            data['2d_coor'][idx]=word_embedding_avg.tolist()

        else:
            tempString = data['data_string'][idx]
            sentenceVectors = model.encode_sentences([tempString], combine_strategy="mean")
            data['2d_coor'][idx]=sentenceVectors.tolist()
            
    return data

dataFrame=updateDataFrame2()
dataFrame.to_csv("submission.csv")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTextRepresentation: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTextRepresentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTextRepresentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/var/folders/2k/s5r6t0jx06n0jzv1j1d39ns00000gr/T/ipykernel_21229/3873087452.py:48: SettingWithCopyWarning: 

317


/var/folders/2k/s5r6t0jx06n0jzv1j1d39ns00000gr/T/ipykernel_21229/3873087452.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['2d_coor'][idx]=word_embedding_avg.tolist()


383
1067
1719
1777
1882
3196
4135
4893
5161
5769
6308
6364
6414
6446
6468
6498
6527
6544
6680
8632
9266


In [6]:
import pandas as pd
dd = pd.read_csv("submission.csv")

In [9]:
dd['2d_coor'].iloc[0]

'[[0.2342969924211502, 0.3045191466808319, 0.1893640011548996, -0.26722076535224915, 0.15332986414432526, -0.4534800350666046, 0.5038731098175049, 0.3504864275455475, -0.2119073122739792, -0.0011940026888623834, -0.18406665325164795, -0.726574182510376, -0.015324123203754425, 0.43473076820373535, 0.1174512431025505, 0.12401726096868515, -0.03527076914906502, 0.12592275440692902, 0.4698863923549652, 0.574870765209198, 0.23317213356494904, 0.029481062665581703, -0.12450814247131348, 0.16203907132148743, 0.19522935152053833, 0.07494428008794785, -0.06592591851949692, 0.17608685791492462, -0.3204493224620819, -0.2634330689907074, 0.09416552633047104, 0.028875941410660744, -0.1690378338098526, 0.37216588854789734, 0.030332056805491447, 0.032980531454086304, 0.537462055683136, -0.014275801368057728, -0.24521677196025848, 0.2975105941295624, -0.11777127534151077, -0.235340878367424, 0.4726163446903229, -0.0961449146270752, -0.09680449217557907, -0.6844377517700195, 0.2555982172489166, -0.1254

In [1]:
# Uses "bert-base-uncased" to get embeddings
# For words more than 500, divides in chunks and gets average embeddings

# Prints first 500 rows in dataframe

import transformers
import torch
import torchvision
import numpy as np
from simpletransformers.language_representation import RepresentationModel

model=RepresentationModel(model_type="bert", model_name="bert-base-uncased", use_cuda=False)

def updateDataFrame2(csv_file_name='Care_review_full.csv', model_max_length=500):
    import pandas as pd
    df = pd.read_csv(csv_file_name)
    data=df.head(500)
    #data = data.drop(columns=['Unnamed: 0'])
    data=data.astype({'2d_coor': 'object'})
    #print(data.dtypes)
    
    def split_sentences(list_of_words):
        chunked_list = list()
        chunk_size = model_max_length
        for i in range(0, len(list_of_words), chunk_size):
            chunked_list.append(list_of_words[i:i+chunk_size])

        return(chunked_list)

    for idx in range(len(data['data_string'])):
        chunkedListWithSent=[]
        sent=""

    
        if len(str(data['data_string'][idx]).split()) > 500:
            tempString = data['data_string'][idx]
            tempStringSplit = tempString.split()
            chunkedLists = split_sentences(tempStringSplit)
            
            for sentenceList in chunkedLists:
                sent=" ".join(sentenceList)
                chunkedListWithSent.append(sent)
                sent=""
             
            sentenceVectors = model.encode_sentences(chunkedListWithSent, combine_strategy="mean")
            word_embedding_avg = np.mean(sentenceVectors, axis=0)

            data['2d_coor'][idx]=word_embedding_avg.tolist()

        else:
            tempString = data['data_string'][idx]
            sentenceVectors = model.encode_sentences([tempString], combine_strategy="mean")
            data['2d_coor'][idx]=sentenceVectors.tolist()
            
    return data

dataFrame=updateDataFrame2()
print(dataFrame)
dataFrame.to_csv("submission.csv")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTextRepresentation: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTextRepresentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTextRepresentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
C:\Users\HP\AppData\Local\Temp\ipykernel_17364\2452095990.py:55: SettingWithCopyWarning: 
A value is trying

317


C:\Users\HP\AppData\Local\Temp\ipykernel_17364\2452095990.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['2d_coor'][idx]=word_embedding_avg.tolist()


383
                                  data_id  \
0    a224545f-1a07-47eb-96ee-a0d0cab23100   
1    01a8cfc8-e207-4c72-a9fa-fa6353fde529   
2    1a4fb4a6-6c56-4610-a151-b4b25a774cc8   
3    9356a4fd-54fd-4604-a6a8-2d3067eba7cc   
4    77526eed-5ff3-4f2f-80ce-da1048137cf4   
..                                    ...   
495  4ab54e0d-a655-489d-8de6-0fadffe72fef   
496  f6578787-c6ea-432c-9a45-26c91a923bc2   
497  983efbd4-88d3-49d5-896b-77d995b68e32   
498  f4374484-6700-4d38-9c9d-1801258d38af   
499  c82129a4-3c7a-4f75-9b15-797e6b59cc26   

                                           data_string  \
0                                  Good place to visit   
1    Went here for a swollen Jaw. Even though I was...   
2    I was seen relatively quickly and the staff wa...   
3    Reception and service couldn't have been more ...   
4    I came in they were very busy the receptionist...   
..                                                 ...   
495  Staff is vey friendly and prompt with servic